In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.getenv("groq_api_key")

In [2]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [68]:
prompt_s = """You are a good SOAP note generator who specicailly generate Subjective, 'S' part. 
                  You will recieve a conversation between a family physician and a patient.
                  You will generate the Subjective part of the SOAP note based on that. Prioritize accuracy and precision. 
                  Do not make up any information. Use format language
                  In the begining, print the patient name three times in three serparated lines"""

In [69]:
template_s = ChatPromptTemplate.from_messages([
    ("system", prompt_s),
    ("human", "{conversation}"),
])

In [58]:
llm = ChatGroq(groq_api_key=groq_api_key, temperature=0)

In [72]:
generate_subjective = template_s | llm
    

In [81]:
prompt_o = """You are a good SOAP note generator who specifically generate Objective / 'O' part. 
                  You will recieve a conversation between a family physician and a patient and a Subjective session of the SOAP notes.
                  You will generate the Objective part of the SOAP note based on the received information. Prioritize accuracy and precision. 
                  If there is no objective information in the input, you cannot return 'No objective information'. Make this part short and simple 
                  Do not make up any information. Use format language. Finally, make the output final version of Objective part that the physician
                  do not need to change anything"""

In [93]:
template_o = ChatPromptTemplate.from_messages([
    ("system", prompt_o),
    ("human", "{conversation}" + 
     "\n {subjective}")
])

In [94]:
generate_objective = template_o | llm

In [44]:
with open("conver.txt", "r") as file:
    conversation = file.read()
    print(conversation)


Doctor: Good morning, Mr. Adams! How have you been feeling lately?

Patient: Morning, Doc. Iâ€™ve been alright, but Iâ€™ve had some concerns lately. Iâ€™ve been feeling really tired, and my energy levels just arenâ€™t what they used to be.

Doctor: I see. How long have you been feeling this way?

Patient: Maybe for the last month or so. At first, I thought it was just work stress, but even on weekends, I feel drained.

Doctor: Got it. Have you noticed any other symptomsâ€”like trouble sleeping, weight changes, or headaches?

Patient: Actually, yes. Iâ€™ve been having trouble sleeping. I wake up in the middle of the night and canâ€™t fall back asleep. And I think Iâ€™ve lost a few pounds, but I havenâ€™t changed my diet.

Doctor: Alright. Have there been any changes in your daily routineâ€”more stress, dietary changes, or any new medications?

Patient: Not really. Work has been a little hectic, but nothing unusual. And Iâ€™m not on any new medications.

Doctor: Understood. Iâ€™d like to

In [48]:
print(type(conversation))

<class 'str'>


In [85]:
response = generate_objective.invoke({"conversation": conversation})

In [86]:
print(response.content)

Objective:

* Patient reports feeling fatigued for the past month.
* Associated symptoms include insomnia and unintentional weight loss.
* No significant changes in daily routine, diet, or medications.
* Vital signs within normal limits.
* Plan: Order blood tests to check for anemia, thyroid issues, and blood sugar levels.


In [87]:
prompt_a = """You are a good SOAP note generator who specifically generate Assessment / 'A' part. 
                  You will recieve a conversation between a family physician and a patient, the Subjective and the Objective sessions of the SOAP notes.
                  You will generate the Assessment part of the SOAP note based on the received information. Prioritize accuracy and precision. 
                  Do not make up any information. Use format language. Finally, make the output final version of Objective part that the physician
                  do not need to change anything"""

In [97]:
template_a = ChatPromptTemplate.from_messages([
    ("system", prompt_o),
    ("human", "{conversation}"
     "\n {subjective}" +
     "\n {objective}")
])

In [98]:
generate_assessment = template_a | llm

In [90]:
prompt_p = """You are a good SOAP note generator who specifically generate Plan / 'P' part. 
                  You will recieve a conversation between a family physician and a patient, the Subjective, the Objective and the Assessment sessions of the SOAP notes.
                  You will generate the Plan part of the SOAP note based on the received information. Prioritize accuracy and precision. 
                  Do not make up any information. Use format language. Finally, make the output final version of Objective part that the physician
                  do not need to change anything"""

In [99]:
template_a = ChatPromptTemplate.from_messages([
    ("system", prompt_o),
    ("human", "{conversation}" + 
     "\n {subjective}" + 
     "\n {objective}" + 
     "\n {assessment}")
])

In [100]:
generate_plan = template_a | llm